# Question 04 Take 2

This implementation is somehow extremely slow. Need to investigate.

# Setup

In [1]:
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
import org.apache.spark.sql.SaveMode
import java.time.temporal.ChronoUnit
import java.time.{Period, LocalDate, Instant}
import java.sql.Timestamp

### Spark parition control based on core availability

In [2]:
val NUM_CORES = 4
val NUM_PARTITIONS = 3

lazy val spark: SparkSession = SparkSession.builder()
    .master("local")
    .appName("flight")
    .getOrCreate()

spark.conf.set("spark.sql.shuffle.partitions", NUM_CORES * NUM_PARTITIONS)
spark.conf.set("spark.default.parallelism", NUM_CORES * NUM_PARTITIONS)

import spark.implicits._

NUM_CORES = 4
NUM_PARTITIONS = 3
spark = <lazy>


<lazy>

## Constants

In [3]:
val CSV_DELIMITER = ","
val FLIGHTDATA_CSV_PATH = "../resources/flightData.csv"
val PASSENGER_CSV_PATH = "../resources/passengers.csv"

val DATE_FORMAT = "yyyy-MM-dd"
val FLIGHT_DATE_FROM = "2017-01-01"
val FLIGHT_DATE_TO   = "2017-12-31"
val NUM_FLIGHT_TOGETHER = 3

val RESULT_DIR = "results/flightsTogether"

val DEBUG = true

CSV_DELIMITER = ,
FLIGHTDATA_CSV_PATH = ../resources/flightData.csv
PASSENGER_CSV_PATH = ../resources/passengers.csv
DATE_FORMAT = yyyy-MM-dd
FLIGHT_DATE_FROM = 2017-01-01
FLIGHT_DATE_TO = 2017-12-31
NUM_FLIGHT_TOGETHER = 3
RESULT_DIR = results/flightsTogether
DEBUG = true


true

# Tools

### Elapsed time profiler

In [4]:
import scala.collection.mutable.ListBuffer

val timing = new StringBuffer
val times = new ListBuffer[Long]()

def clear(): Unit = {
    timing.setLength(0)
    times.clear
}
def average(): Long = {
    times.reduce(_+_) / times.length
}

/**
@param label Description about the run
@code code to execute
@return execution
*/
def timed[T](label: String, code: => T): T = {
    val start = System.currentTimeMillis()
    val result = code
    val stop = System.currentTimeMillis()
    timing.append(s"Processing $label took ${stop - start} ms.\n")
    times.append(stop - start)
    code
}

timing = 
times = ListBuffer()


clear: ()Unit
average: ()Long
timed: [T](label: String, code: => T)T


ListBuffer()

In [5]:
// To flush out error: missing argument list for method timed
println("")

<console>:45: error: missing argument list for method timed
Unapplied methods are only converted to functions when a function type is expected.
You can make this conversion explicit by writing `timed _` or `timed(_,_)` instead of `timed`.
       timed
       ^
lastException: Throwable = null


### Save to file

In [6]:
def save(df: DataFrame) = {
    df.coalesce(1)
    .write
    .format("csv")
    .mode(SaveMode.Overwrite)
    .option("header", "true")
    .save(RESULT_DIR)
}

save: (df: org.apache.spark.sql.DataFrame)Unit


# Main

### Base FlightData DataFrame

In [7]:
// Transformations, no action yet
val flightData = spark.read.format("csv")
    .option("header", "true")
    .option("delimiter", ",")
    .option("dateFormat", "yyyy-MM-dd")
    .option("inferSchema", "true")
    .load(FLIGHTDATA_CSV_PATH)
    .select(
        "passengerId",
        "flightId",
        "date"
    )

flightData = [passengerId: int, flightId: int ... 1 more field]


[passengerId: int, flightId: int ... 1 more field]

### SQL

In [8]:
def queryFlightsTogether(atLeastNTimes: Int, fromDate: String, toDate: String): String = {
    val query = s"""
    WITH 
        --------------------------------------------------------------------------------
        -- Passengers flew more than atLeastNTimes times.
        --------------------------------------------------------------------------------
        more_than_n_flights AS (
            SELECT passengerId
            FROM flightData
            GROUP BY passengerId
            HAVING count(flightId) > $atLeastNTimes
            ORDER BY passengerId
        )

    SELECT 
        f.passengerId AS `Passenger 1 ID`, 
        s.passengerId AS `Passenger 2 ID`, 
        count(s.flightId) AS `Number of flights together`,
        '$fromDate' as From,
        '$toDate' as To
    FROM
        flightData f 
        --------------------------------------------------------------------------------
        -- Passengers more than atLeastNTimes flights
        --------------------------------------------------------------------------------
        INNER JOIN more_than_n_flights m 
            ON f.passengerId == m.passengerId
        --------------------------------------------------------------------------------
        -- Passengers who shared same flights
        --------------------------------------------------------------------------------
        INNER JOIN flightData s 
            ON f.flightId == s.flightId
    WHERE
        f.passengerId < s.passengerId AND
        f.date >= to_timestamp('$fromDate', '$DATE_FORMAT') AND
        f.date <= to_timestamp('$toDate',   '$DATE_FORMAT') AND
        s.date >= to_timestamp('$fromDate', '$DATE_FORMAT') AND
        s.date <= to_timestamp('$toDate',   '$DATE_FORMAT')
    GROUP BY 
        f.passengerId, s.passengerId
    HAVING 
        count(s.flightId) > $atLeastNTimes
    --ORDER BY 
    --    f.passengerId, s.passengerId
    """
    query
}

queryFlightsTogether: (atLeastNTimes: Int, fromDate: String, toDate: String)String


## SparkSQL runner

In [9]:
/**
Get the dataframe of |passenger 1|passenger 2|num flights| from | to|
@param atLeastNTimes Number beyond which passengers share flights
@param from Date including and after which the shared flight was scheduled
@param to Data including and before which the shared flight was scheduled
@return DataFrame
*/
def flownTogether(atLeastNTimes: Int, from: String, to: String) = {
    val execution = spark
        .sql(queryFlightsTogether(atLeastNTimes, from, to))
        .sort(
            desc("Number of flights together"), 
            asc("Passenger 1 ID"),
            asc("Passenger 2 ID")
        )
    execution
}

flownTogether: (atLeastNTimes: Int, from: String, to: String)org.apache.spark.sql.Dataset[org.apache.spark.sql.Row]


### Utility to run multiple times and time the execution

In [10]:
/**
Run the SparkSQL 
@param label Lable to describe this run
@param query SQL 
@param repeats Number of run
@return Average execution time in msec
*/
def run(label: String, repeats: Int): Long = {
    flightData
        .persist
        .createOrReplaceTempView("flightData")

    val execution = flownTogether(
        NUM_FLIGHT_TOGETHER,
        FLIGHT_DATE_FROM,
        FLIGHT_DATE_TO
    )
    clear()
    for (i <- (0 until repeats)){
        timed(
            label,
            execution.show(5)
        )
        println(timing)
        println(s"Average time $average ms")
    }
    println(execution.rdd.toDebugString)    
    save(execution)

    if (DEBUG) println(execution.rdd.toDebugString)    

    spark.catalog.dropTempView("flightData")
    flightData.unpersist
    
    average
}

run: (label: String, repeats: Int)Long


## Execution 01
Order by (passengerId)

In [12]:
var df = flightData
    .orderBy("passengerId")

val timeOrderPassenger = run(
    "Order by (passengerId)",
    3
)

+--------------+--------------+--------------------------+----------+----------+
|Passenger 1 ID|Passenger 2 ID|Number of flights together|      From|        To|
+--------------+--------------+--------------------------+----------+----------+
|           701|           760|                        15|2017-01-01|2017-12-31|
|           760|           701|                        15|2017-01-01|2017-12-31|
|          2717|          2759|                        14|2017-01-01|2017-12-31|
|          2759|          2717|                        14|2017-01-01|2017-12-31|
|          3503|          3590|                        14|2017-01-01|2017-12-31|
+--------------+--------------+--------------------------+----------+----------+
only showing top 5 rows

+--------------+--------------+--------------------------+----------+----------+
|Passenger 1 ID|Passenger 2 ID|Number of flights together|      From|        To|
+--------------+--------------+--------------------------+----------+----------+
|  

df = [passengerId: int, flightId: int ... 1 more field]
timeOrderPassenger = 9978


lastException: Throwable = null


[passengerId: int, flightId: int ... 1 more field]